In [ ]:
from src.eeg import struct
from src.core import REc as rec
from src.game import classify_epochs, evaluate_nodes, check_until

from itertools import combinations
from os import listdir, makedirs
import matplotlib.pyplot as plt

In [ ]:
main_folder = "/home/kivi/gdrive/epigame-folder/"

path_cm = main_folder + "connectivity_matrices/" 

In [ ]:
woi = input("Time window:\n 1. Non-seizure (baseline)\n 2. Pre-seizure (5 min prior to seizure)\n 3. Pre-seizure (4 min prior to seizure)\n 4. Pre-seizure (3 min prior to seizure)\n 5. Pre-seizure (2 min prior to seizure)\n 6. Pre-seizure (1 min prior to seizure)\n 7. Transition to seizure (1 min interval)\n 8. Transition to seizure (2 min interval)\n 9. Transition to seizure (60% seizure length interval)\n 10. Seizure\n Indicate a number: ")

woi_code = {'1':"baseline", '2':"preseizure5", '3':"preseizure4", '4':"preseizure3", '5':"preseizure2", '6':"preseizure1", '7':"transition1", '8':"transition2", '9':"transition60", '10':"seizure"}

In [ ]:
max_net_size = 20

In [ ]:
path_net = main_folder + "selected_network/"
makedirs(path_net, exist_ok=True)

In [ ]:
for file_cm in listdir(path_cm):
    print("Processing...")
    subject_id = file_cm.split("/")[-1][0:3]
    print("Connectivity matrices of", file_cm)

    cm = rec.load(path_cm + file_cm).data

    print("\nTotal number of epochs =", len(cm.X))
    print("Connectivity matrix shape =", cm.X[0].shape)

    plt.figure(figsize=(5,5))
    plt.imshow(cm.X[-1], cmap='Blues', interpolation='nearest')
    plt.show()
    print(cm.X[-1])

    nodes = cm.nodes
    node_ids = list(range(len(nodes))) 
    print("Number of nodes =",len(nodes))
    print("\nNodes:", nodes)

    node_pairs = combinations(node_ids, 2)

    print("\nProcessing node combinations...")

    parallelize = Parallel(n_jobs=-1)(delayed(evaluate_nodes)(pair, nodes, classify_epochs(cm, pair)) for pair in node_pairs)
    base = [p for p in parallelize]

    print(f"{len(base)} finished")

    base.sort(key=lambda x:x[-1], reverse=True)
    
    print(f"Best node pair: {base[0][1]}")

    best_score, net_size, possible_node_groups, nets = base[0][-1], 3, base[:], []

    while net_size <= max_net_size:

        print(f"Checking networks with {net_size} nodes...")

        head = check_until(possible_node_groups)
        tests = 0

        for node_group in possible_node_groups[:head if head>0 else 1]:

            for node in node_ids:

                if node not in node_group[0]:

                    test_group = node_group[0] + (node,)

                    nets.append(evaluate_nodes(test_group, nodes, classify_epochs(cm, test_group)))

                    tests += 1

        print(f"Tested {tests} node groups.")

        nets.sort(key=lambda x:x[-1], reverse=True)

        print(f"Best network: {nets[0][1]}")

        evaluation_score = nets[0][-1]

        if evaluation_score >= best_score:

            if net_size <= max_net_size:

                best_score = nets[0][-1]
                possible_node_groups = nets[:]
                nets = []
                               
            net_size += 1
            
        else: break

    selected_net = sorted(set([t for n in nets[:check_until(nets)] for t in n[1].split('<->')]))

    print(f"\nSelected network: {selected_net} ({len(selected_net)} nodes in total)")


    file_net = file_cm.split(".")[1]
    rec(struct(base=base, nets=nets, nodes=selected_net)).save(path_net + f"{file_net}.res")